In [89]:
import torch
import torchvision.transforms as T
from PIL import Image
CHANNELS_TO_MODE = {
    1 : 'L',
    3 : 'RGB',
    4 : 'RGBA'
}

def seek_all_images(img, channels = 3):
    # 从给定的图像中提取所有的图像帧
    assert channels in CHANNELS_TO_MODE, f'channels {channels} invalid'
    # 验证channels参数的有效性。它检查channels是否在CHANNELS_TO_MODE字典中
    mode = CHANNELS_TO_MODE[channels]
    # 将不同的通道数映射到相应的图像模式
    i = 0
    while True:
        try:
            #使用img.convert(mode)将当前帧转换为指定的图像模式，并使用yield关键字将转换后的图像帧作为生成器的输出
            # 生成器可以逐个地获取每一帧图像，而不需要一次性加载整个图像序列到内存中。这对于处理大型图像序列或视频非常有用，可以节省内存并提高效率。
            img.seek(i)
            yield img.convert(mode)
        except EOFError:
            break
        i += 1

# gif -> (channels, frame, height, width) tensor
def gif_to_tensor(path, channels = 3, transform = T.ToTensor()):
    img = Image.open(path)
    tensors = tuple(map(transform, seek_all_images(img, channels = channels)))
    print(len(tensors))
    print(tensors[0].shape)
    return torch.stack(tensors, dim = 1)

# tensor of shape (channels, frames, height, width) -> gif
# 将一个形状为(channels, frames, height, width)的张量转换为GIF图像
def video_tensor_to_gif(tensor, path, duration = 200, loop = 0, optimize = False):      # NOTE changed optimize to False
    """
    tensor：输入的张量，表示视频的像素数据。
    path：保存生成的GIF图像的路径。
    duration：每一帧图像在GIF中的显示时间（以毫秒为单位），默认为200毫秒。
    loop：GIF图像的循环次数，默认为0表示无限循环。
    optimize：是否对图像进行优化，默认为False
    """ 
    images = map(T.ToPILImage(), tensor.unbind(dim = 1)) # 将[3,11,96, 96]的张量转为 11个[3, 96, 96]张量组成的元组，之后再转为11个PIL图像组成的迭代器,此时为RGB格式
    # convert images since optimize = False gives issues with non-Palette images
    if optimize == False:
        images = map(lambda img: img.convert('L').convert('P'), images)
        # 先将其转换为灰度图像（使用img.convert('L')），然后再转换为调色板图像（使用img.convert('P')）。
        # 这是因为当optimize为False时，对非调色板图像进行优化会出现问题。
    first_img, *rest_imgs = images
    # 将第一帧图像保存为GIF图像的第一帧，将剩余的图像作为附加帧
    first_img.save(path, save_all = True, append_images = rest_imgs, duration = duration, loop = loop, optimize = optimize)
    # 使用PIL库中的save()函数来保存图像的代码示例。让我们逐个解释每个参数的作用：

    # path：这是保存图像的文件路径。你需要提供一个有效的文件路径，包括文件名和文件格式后缀。

    # save_all：这是一个布尔值参数，用于指定是否保存所有图像帧。如果设置为True，则会保存所有图像帧；如果设置为False，则只保存第一帧。在这个例子中，save_all被设置为True，表示保存所有图像帧。

    # append_images：这是一个图像列表，用于指定要附加到第一帧后面的其他图像帧。你可以将多个图像帧作为列表传递给append_images参数，这些图像帧将按顺序附加到第一帧后面。

    # duration：这是一个整数或浮点数，用于指定每个图像帧的显示时间（以毫秒为单位）。你可以设置不同的duration值来控制每个图像帧的显示时间长度。

    # loop：这是一个整数参数，用于指定循环播放的次数。默认情况下，循环播放是无限的，即loop值为0。如果你想限制循环播放的次数，可以将loop设置为一个正整数。

    # optimize：这是一个布尔值参数，用于指定是否对图像进行优化。如果设置为True，则会尝试对图像进行优化以减小文件大小；
    return images

In [ ]:
# 做实验验证PIL的optimise和所给代码的optimize有什么区别，以及为什么要先转为灰度图像再转为调色板图像，
# 这是因为当optimize为False时，对非调色板图像进行优化会出现问题。
# 直接使用

In [12]:
path = '/root/VideoMetamaterials/data/lagrangian/training/gifs/ener/0.gif'
ener_0 = gif_to_tensor(path)
ener_0.shape

11
torch.Size([3, 96, 96])


torch.Size([3, 11, 96, 96])

In [91]:
video_tensor_to_gif(ener_0,'./0.gif',optimize =True)

In [92]:
img = Image.open('./0.gif')
img.mode # 为啥不是P模式呢？

'P'

In [21]:
from PIL import Image
import numpy as np
# img = Image.open('./0.gif')
img = Image.open(path)
print(img.mode)  # "1-二值图"，“L-灰度图”，“RGB-三原色”"RGBA-三原色+透明度alpha", "P"调色板图像
# RGB采用R，G，B三个通道，每个通道在每一个像素位置用1个字节即8个比特存储对应通道的颜色值（0~255）
print(np.array(img.getpalette()).reshape(256,3).shape)  # 调色盘存储在图像文件的文件头部附近: 256个配色方案*3个通道值的格式
# 使用了调色盘的图像将会被单通道存储，每个像素位置的值是调色盘“表”中的索引，这在存储图像的时候空间要求从RGB的3个字节变成了1个字节

# "调色板图像"是一种特殊的图像格式，它使用一个颜色索引表或称为"调色板"来表示图像中的颜色。每个像素在图像数据中不是直接存储颜色信息，而是存储一个索引值，这个索引值对应调色板中的一个颜色。

# 例如，如果一个图像是8位的调色板图像，那么它的每个像素都是一个0到255的值，这个值是调色板中的索引。调色板本身是一个256*3的数组，每个索引对应一个RGB颜色值。

# 调色板图像的优点是可以大大减少图像的存储空间，特别是对于颜色种类较少的图像，如动漫、图标等。但是，它的缺点是颜色的种类有限，不能精确表示高色深的图像。

# 在Python的PIL库中，可以使用`Image.convert('P')`方法将图像转换为调色板图像。

P
(256, 3)


In [ ]:
def load_gifs(folder):
    # load ener data
    ener_folder = folder + 'gifs/ener/'
    self.paths_ener = [p for ext in exts for p in Path(f'{ener_folder}').glob(f'**/*.{ext}')]
    # sort paths by number of name
    self.paths_ener = sorted(self.paths_ener, key=lambda x: int(x.name.split('.')[0]))
    assert all([int(p.stem) == i for i, p in enumerate(self.paths_ener)]), 'file position is not equal to index'

    assert len(self.paths_ener) == len(self.paths_top), 'number of files in fields and top folders are not equal.'